In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import pandas as pd
import time
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
#url 내에서 상세 정보 크롤링 해보기
options = webdriver.ChromeOptions()
#options.headless = True
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
browser = webdriver.Chrome(options=options)
browser.maximize_window()

txt = pd.read_csv('./LCK_2019_Spring_Promotion_match_links.txt', header=None)
url_list = [i for i in txt.iloc[:,0]]
url = url_list[0]

browser.get(url)

In [3]:
#경기일
date = url[-10:]

date

'2018-09-22'

In [4]:
#사이드창 닫기
left_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[1]/div')
if left_close_button.get_attribute('innerText') == '닫기':
    left_close_button.click()
    
right_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[2]/div')
if right_close_button.get_attribute('innerText') == '닫기':
    right_close_button.click()

In [5]:
#매치 스코어
time.sleep(1)

match_score = browser.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div/div/div[1]/div[1]/div[2]').get_attribute('innerText').split('\n')
match_score = match_score[0] + '(' + match_score[1] + ')' + ' : ' + match_score[3] + '(' + match_score[2] + ')'

match_score 

'MVP(0) : BtC(3)'

In [6]:
#세트 선택
set_list = browser.find_elements(By.CLASS_NAME, 'shrink-0')  

set_1 = set_list[0]
set_1.click()

In [7]:
set_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[1]').get_attribute('innerText').split('\n')

#경기 시간
set_time = set_data[0]

#팀명
team1_name = set_data[1].upper()
team2_name = set_data[-1].upper()

set_time, team1_name, team2_name

('39:50', 'MVP', 'BTC')

In [8]:
#경기별 하이라이트 영상 url(없는 경우도 있음)
try:
    videos = browser.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div/div/div[1]/div[2]').find_elements(By.TAG_NAME, 'li')
    video_url_list = []
    for video in videos:
        video.click()
        video_url_list.append(browser.find_element(By.TAG_NAME, 'iframe').get_attribute('src'))

    close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div/div/div[1]/div[2]/div/div[2]/button')
    close_button.click()
except:
    video_url_list = None

print(video_url_list)

['https://www.youtube.com/embed/Ro3fI5U0fO8', 'https://www.youtube.com/embed/gTI7vo_a8DI', 'https://www.youtube.com/embed/O7WC58JYISQ', 'https://www.youtube.com/embed/DJjf0vdpA9c', 'https://www.youtube.com/embed/0Lc-1SvSPI4', 'https://www.youtube.com/embed/pNrgoPvpQuQ']


In [9]:
#밴(보통 5개이지만, 그렇지 않은 경우도 있음)
team1_banned = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[1]/div[1]/ul[2]')
team1_banned = team1_banned.find_elements(By.TAG_NAME, 'a')
team1_banned = [banned_champ.find_elements(By.TAG_NAME, 'img') for banned_champ in team1_banned]

team2_banned = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[2]/div[1]/ul[2]')
team2_banned = team2_banned.find_elements(By.TAG_NAME, 'a')
team2_banned = [banned_champ.find_elements(By.TAG_NAME, 'img') for banned_champ in team2_banned]

team1_banned = [banned_champ[0].get_attribute('src') for banned_champ in team1_banned]
team2_banned = [banned_champ[0].get_attribute('src') for banned_champ in team2_banned]

team1_banned = [str(banned_champ.split(sep='.')[-2]).split(sep='/')[-1] for banned_champ in team1_banned]
team2_banned = [str(banned_champ.split(sep='.')[-2]).split(sep='/')[-1] for banned_champ in team2_banned]

team1_banned, team2_banned

(['DrMundo', 'Irelia', 'Shen', 'Varus', 'Aatrox'],
 ['Urgot', 'Sion', 'Gragas', 'Gnar', 'Ornn'])

In [10]:
#획득 골드
team1_gold = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[2]/span[2]').get_attribute('innerText')
team2_gold = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[2]/span[3]').get_attribute('innerText')

team1_gold, team2_gold

('62,214', '75,680')

In [11]:
#오브젝트(타워, 억제기, 전령, 드래곤, 장로드래곤, 바론)
team1_object = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[1]/div[2]')
team1_object = team1_object.text.split(sep='\n')[1:]

team2_object = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[2]/div[2]')
team2_object = team2_object.text.split(sep='\n')[1:]

team1_tower, team1_turret, team1_herald, team1_dragon, team1_elderly_dragon, team1_baron = [team1_object[i] for i in range(6)]
team2_tower, team2_turret, team2_herald, team2_dragon, team2_elderly_dragon, team2_baron = [team2_object[i] for i in range(6)]

print((team1_tower, team1_turret, team1_herald, team1_dragon, team1_elderly_dragon, team1_baron))
print((team2_tower, team2_turret, team2_herald, team2_dragon, team2_elderly_dragon, team2_baron))

('2', '0', '1', '0', '0', '0')
('10', '2', '0', '5', '1', '1')


In [12]:
#KDA(kill/death/assist)
team1_kda = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[1]/span[2]').get_attribute('innerText')
team2_kda = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[3]/span[2]').get_attribute('innerText')

team1_k, team1_d, team1_a = [team1_kda.split(' / ')[i] for i in range(3)]
team2_k, team2_d, team2_a = [team2_kda.split(' / ')[i] for i in range(3)]

print((team1_k, team1_d, team1_a))
print((team2_k, team2_d, team2_a))

('6', '12', '13')
('12', '6', '31')


In [13]:
#진영(blue/red)
data = team1_side = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').get_attribute('innerText').split('\n\t\n')
team1_side = data[:len(data)//2 ][0].split('\n')[2][:-5]
team2_side = data[len(data)//2:][0].split('\n')[2][:-5]

team1_side, team2_side

('Blue', 'Red')

In [14]:
#승패 여부(victory/defeated)
xpath = "//strong[contains(text(), 'Victory')]/.."   #승리팀 찾기
xpath = browser.find_element(By.XPATH, xpath)
victory_team = xpath.get_attribute('innerText')
            
if team1_name == victory_team[0]:
    team1_v_d, team2_v_d = 'Victory', 'Defeat'
else:
    team1_v_d, team2_v_d = 'Defeat', 'Victory'
    
team1_v_d, team2_v_d

('Defeat', 'Victory')

In [15]:
#패치 버전
xpath = "//div[contains(text(), 'Ver.')]"
version = browser.find_element(By.XPATH, xpath).get_attribute('innerText')[5:]
version

'8.17'

In [16]:
browser.quit()

In [17]:
data = []

data.append([date, version, match_score, 1, team1_name, team1_v_d, team1_side, team1_banned, team1_gold, 
             team1_tower, team1_turret, team1_herald, team1_dragon, team1_elderly_dragon, team1_baron, team1_k, team1_d, team1_a, video_url_list])
data.append([date, version, match_score, 1, team2_name, team2_v_d, team2_side, team2_banned, team2_gold, 
             team2_tower, team2_turret, team2_herald, team2_dragon, team2_elderly_dragon, team2_baron, team2_k, team2_d, team2_a, video_url_list])

columns = ['date', 'version', 'match_score', 'set', 'team', 'win/lose', 'side', 'ban', 'gold', 
           'tower','turret', 'herald', 'dragon', 'elderly_dragon', 'baron',
           'k', 'd', 'a', 'videos']
pd.DataFrame(data, columns=columns).set_index('date')

,version,match_score,set,team,win/lose,side,ban,gold,tower,turret,herald,dragon,elderly_dragon,baron,k,d,a,videos
date,,,,,,,,,,,,,,,,,,
2018-09-22,8.17,MVP(0) : BtC(3),1,MVP,Defeat,Blue,"[DrMundo, Irelia, Shen, Varus, Aatrox]","62,214",2,0,1,0,0,0,6,12,13,"[https://www.youtube.com/embed/Ro3fI5U0fO8, ht..."
2018-09-22,8.17,MVP(0) : BtC(3),1,BTC,Victory,Red,"[Urgot, Sion, Gragas, Gnar, Ornn]","75,680",10,2,0,5,1,1,12,6,31,"[https://www.youtube.com/embed/Ro3fI5U0fO8, ht..."


In [18]:
#현재 디렉토리 내 txt 파일 모두 불러오기(리그, 연도별)
def get_txt_files(path, league, year):
    txt_list = [file for file in os.listdir(path) if (file.endswith('txt') and file.startswith(f'{league}_{year}'))]
    return txt_list

In [19]:
#txt 파일 내 모든 url 크롤링하기
def crawling_team_data(txt_list): 
    from tqdm import tqdm
    final_data = []
    failed_url = []
    
    total_time = 0
    
    for txt in txt_list:    
        start_time = time.time()   #소요시간 체크
        
        urls = pd.read_csv(txt, header=None)
        url_list = [i for i in urls.iloc[:,0]]
        
        for url_idx, url in enumerate(tqdm(url_list, desc=txt)):                
            try:
                options = webdriver.ChromeOptions()
                options.headless = True
                options.add_argument('window_size=1920x1080')
                options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
                browser = webdriver.Chrome(options=options)
                browser.maximize_window()
                browser.get(url)
                
                time.sleep(1)
                
                #사이드창 닫기
                left_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[1]/div')
                if left_close_button.get_attribute('innerText') == '닫기':
                    left_close_button.click()
                    
                right_close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[3]/div[2]/div')
                if right_close_button.get_attribute('innerText') == '닫기':
                    right_close_button.click()
    
                #대회 정보
                tournament_name = txt[:-16]
                
                #경기일
                date = url[-10:]    
                
                #패치 버전
                xpath = "//div[contains(text(), 'Ver.')]"
                version = browser.find_element(By.XPATH, xpath).get_attribute('innerText')[5:]
                
                time.sleep(1)
                #매치 스코어
                match_score = browser.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div/div/div[1]/div[1]/div[2]').get_attribute('innerText').split('\n')
                match_score = match_score[0] + '(' + match_score[1] + ')' + ' : ' + match_score[3] + '(' + match_score[2] + ')'
                
                #세트 선택
                set_list = browser.find_elements(By.CLASS_NAME, 'shrink-0')  
                for set_num in range(len(set_list)):
                    current_set = set_list[set_num]
                    current_set.click()
                    
                    time.sleep(1)
                    set_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[1]').get_attribute('innerText').split('\n')
                    
                    #팀명
                    team1_name = set_data[1]
                    team2_name = set_data[-1]
                    
                    #경기 시간
                    set_time = set_data[0]
                    
                    #진영(blue/red)
                    data = team1_side = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').get_attribute('innerText').split('\n\t\n')
                    team1_side = data[:len(data)//2 ][0].split('\n')[2][:-5]
                    team2_side = data[len(data)//2:][0].split('\n')[2][:-5]
                    
                    #경기영상 url
                    try:
                        videos = browser.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div/div/div[1]/div[2]').find_elements(By.TAG_NAME, 'li')
                        video = videos[set_num]
                        video.click()
                        url = browser.find_element(By.TAG_NAME, 'iframe').get_attribute('src')

                        close_button = browser.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div/div/div[1]/div[2]/div/div[2]/button')
                        close_button.click()
                    except:
                        url = None
                            
                    #밴
                    team1_banned = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[1]/div[1]/ul[2]')
                    team1_banned = team1_banned.find_elements(By.TAG_NAME, 'a')
                    team1_banned = [banned_champ.find_elements(By.TAG_NAME, 'img') for banned_champ in team1_banned]

                    team2_banned = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[2]/div[1]/ul[2]')
                    team2_banned = team2_banned.find_elements(By.TAG_NAME, 'a')
                    team2_banned = [banned_champ.find_elements(By.TAG_NAME, 'img') for banned_champ in team2_banned]

                    team1_banned = [banned_champ[0].get_attribute('src') for banned_champ in team1_banned]
                    team2_banned = [banned_champ[0].get_attribute('src') for banned_champ in team2_banned]

                    team1_banned = [str(banned_champ.split(sep='.')[-2]).split(sep='/')[-1] for banned_champ in team1_banned]
                    team2_banned = [str(banned_champ.split(sep='.')[-2]).split(sep='/')[-1] for banned_champ in team2_banned]
                    
                    #획득 골드
                    team1_gold = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[2]/span[2]').get_attribute('innerText')
                    team2_gold = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[2]/span[3]').get_attribute('innerText')
                    
                    #오브젝트(타워, 억제기, 전령, 드래곤, 장로드래곤, 바론)
                    team1_object = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[1]/div[2]')
                    team1_object = team1_object.text.split(sep='\n')[1:]

                    team2_object = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[2]/div[2]')
                    team2_object = team2_object.text.split(sep='\n')[1:]

                    team1_tower, team1_turret, team1_herald, team1_dragon, team1_elderly_dragon, team1_baron = [team1_object[i] for i in range(6)]
                    team2_tower, team2_turret, team2_herald, team2_dragon, team2_elderly_dragon, team2_baron = [team2_object[i] for i in range(6)]
                    
                    #KDA(kill/death/assist)
                    team1_kda = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[1]/span[2]').get_attribute('innerText')
                    team2_kda = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[3]/span[2]').get_attribute('innerText')

                    team1_k, team1_d, team1_a = [team1_kda.split(' / ')[i] for i in range(3)]
                    team2_k, team2_d, team2_a = [team2_kda.split(' / ')[i] for i in range(3)]
                    
                    #승패 여부(victory/defeated)
                    xpath = "//strong[contains(text(), 'Victory')]/.."   #승리팀 찾기
                    xpath = browser.find_element(By.XPATH, xpath)
                    victory_team = xpath.get_attribute('innerText')
                    
                    if team1_name == victory_team[0]:
                        team1_v_d, team2_v_d = 'Victory', 'Defeat'
                    else:
                        team1_v_d, team2_v_d = 'Defeat', 'Victory'

                    final_data.append([tournament_name, date, version, match_score, set_num+1, set_time, team1_name, team1_v_d, team1_side, team1_banned, team1_gold, 
                                team1_tower, team1_turret, team1_herald, team1_dragon, team1_elderly_dragon, team1_baron, team1_k, team1_d, team1_a, url])
                    final_data.append([tournament_name, date, version, match_score, set_num+1, set_time, team2_name, team2_v_d, team2_side, team2_banned, team2_gold, 
                                team2_tower, team2_turret, team2_herald, team2_dragon, team2_elderly_dragon, team2_baron, team2_k, team2_d, team2_a, url])
                 
            except:
                failed_url.append(url)
                
        end_time = time.time()
        crawling_time = int(end_time - start_time)
        total_time += crawling_time   
        if len(failed_url) == 0:
            print('크롤링을 정상적으로 완료했습니다!')
        else:
            print(f'실패한 url : {failed_url}')
            
    print(f'총 소요 시간 : {round(total_time / 60, 2)}분')    
    print('-'*100)  
    
    columns = ['tournament_name', 'date', 'version', 'match_score', 'set', 'time', 'team', 'win/lose', 'side', 'ban', 'gold', 
           'tower','turret', 'herald', 'dragon', 'elderly_dragon', 'baron',
           'k', 'd', 'a', 'video_url']

    df = pd.DataFrame(final_data, columns=columns).set_index(['tournament_name', 'date', 'version', 'match_score', 'set', 'time', 'team'])    
    
    return df        

In [20]:
#함수 테스트
txt_list = get_txt_files('./', 'LCK', '2022')   
print(f'LCK txt 파일 개수 : {len(txt_list)}')

txt_list

LCK txt 파일 개수 : 3


['LCK_2022_Spring_Playoffs_match_links.txt',
 'LCK_2022_Spring_Regular season_match_links.txt',
 'LCK_2022_Summer_Regular Season_match_links.txt']

In [21]:
df = crawling_team_data(txt_list[:1])  #우선 txt파일 하나만 사용

print(df.shape)
df

LCK_2022_Spring_Playoffs_match_links.txt: 100%|██████████| 5/5 [01:27<00:00, 17.53s/it]


크롤링을 정상적으로 완료했습니다!
총 소요 시간 : 1.45분
----------------------------------------------------------------------------------------------------
(40, 14)


win/lose  \
tournament_name          date       version match_score     set time  team                  
LCK_2022_Spring_Playoffs 2022-04-02 12.5    T1(3) : GEN(1)  1   30:09 GEN          Defeat   
                                                                      T1          Victory   
                                                            2   35:38 GEN          Defeat   
                                                                      T1          Victory   
                                                            3   37:27 T1           Defeat   
                                                                      GEN         Victory   
                                                            4   27:55 GEN          Defeat   
                                                                      T1          Victory   
                         2022-03-27 12.5    GEN(3) : DK(2)  1   38:20 GEN          Defeat   
                                                                      DK deokdam  Victory   
                                                            2   37:27 DK           Defeat   
                                                                      GEN Chovy   Victory   
                                                            3   35:20 GEN          Defeat   
                                                                      DK Hoya     Victory   
                                                            4   33:03 GEN          Defeat   
                                                                      GEN Chovy   Victory   
                                                            5   34:06 DK           Defeat   
                                                                      GEN Chovy   Victory   
                         2022-03-26 12.5    T1(3) : KDF(0)  1   26:17 T1           Defeat   
                                                                      T1 Faker    Victory   
                                                            2   20:16 KDF          Defeat   
                                                                      T1 Keria    Victory   
                                                            3   31:58 KDF          Defeat   
                                                                      T1 Keria    Victory   
                         2022-03-24 12.5    DRX(2) : KDF(3) 1   37:27 DRX          Defeat   
                                                                      KDF Teddy   Victory   
                                                            2   40:17 DRX          Defeat   
                                                                      DRX Zeka    Victory   
                                                            3   35:10 KDF          Defeat   
                                                                      KDF Kiin    Victory   
                                                            4   30:34 DRX          Defeat   
                                                                      DRX Pyosik  Victory   
                                                            5   40:02 KDF          Defeat   
                                                                      KDF Teddy   Victory   
                         2022-03-23 12.5    DK(3) : BRO(0)  1   39:54 DK           Defeat   
                                                                      DK deokdam  Victory   
                                                            2   32:35 BRO          Defeat   
                                                                      DK deokdam  Victory   
                                                            3   26:55 DK           Defeat   
                                                                      DK Burdol   Victory   

                                                                                  side  \
tournament_name          date       version match_score     set time  team          

## 리그/연도별 전체 데이터 크롤링

In [22]:
#리그/연도별 크롤링 후 csv 파일로 만들기
leagues = ['LCK', 'LPL', 'LCS', 'LEC']
years = [str(year) for year in range(2016, 2023)]

for league in leagues:
    for year in years:
        txt_list = get_txt_files('./', league, year)
        print(f'{league}_{year} txt 파일 개수 : {len(txt_list)}')

        df = crawling_team_data(txt_list)
        df.to_csv(f'{league}_{year}_team_data.csv')

LCK_2016 txt 파일 개수 : 8


LCK_2016_Spring_Playoffs_match_links.txt:  75%|███████▌  | 3/4 [01:01<00:21, 21.25s/it]